In [48]:
import os
import numpy as np
import json
import pandas as pd
import datetime as DT
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Get required data
From the github repository

In [47]:
# Clone the entire repo.
!git clone -l -s https://github.com/XLipcak/rev-rec.git cloned-repo

fatal: destination path 'cloned-repo' already exists and is not an empty directory.


In [48]:
%ls

cloned-repo/  drive/  sample_data/


In [49]:
# Save OpenStack dataset
import shutil
src = "/content/cloned-repo/rev-rec-data/openstack.json"
dst = "/content/drive/MyDrive/Colab Notebooks/My Project/openstack.json"
shutil.move(src, dst)

FileNotFoundError: ignored

# Read json file

In [110]:
f = open('/content/drive/MyDrive/Colab Notebooks/My Project/openstack.json',)
data = json.load(f)  # returns json object as a dictionary
for i in range(5):   # 5 first elements of the json list
    print(data[i])
f.close()

{'subProject': 'openstack/tempest', 'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d', 'changeNumber': 6139, 'timestamp': 1333428006000, 'reviewers': [{'accountId': '97', 'email': 'daryl.walleck@rackspace.com', 'name': 'Daryl Walleck', 'avatar': None}, {'accountId': '7', 'email': 'jaypipes@gmail.com', 'name': 'Jay Pipes', 'avatar': None}], 'owner': {'accountId': '2238', 'email': 'rajalakshmi.girish@hpe.com', 'name': 'rajalakshmi-ganesan', 'avatar': None}, 'filePaths': [{'location': 'tempest/tests/test_volumes_get.py'}]}
{'subProject': 'openstack/tempest', 'changeId': 'I10e7116570f922ec87e23b1f880cd4b1c08c3088', 'changeNumber': 7438, 'timestamp': 1337072165000, 'reviewers': [{'accountId': '97', 'email': 'daryl.walleck@rackspace.com', 'name': 'Daryl Walleck', 'avatar': None}, {'accountId': '7', 'email': 'jaypipes@gmail.com', 'name': 'Jay Pipes', 'avatar': None}], 'owner': {'accountId': '4120', 'email': 'rohit.karajgi@ril.com', 'name': 'Rohit Karajgi', 'avatar': None}, 'filePaths': 

## Hands on with given data

In [111]:
type(data)

list

In [112]:
len(data)

6545

In [113]:
# get an element of the list of data
data[0]

{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d',
 'changeNumber': 6139,
 'filePaths': [{'location': 'tempest/tests/test_volumes_get.py'}],
 'owner': {'accountId': '2238',
  'avatar': None,
  'email': 'rajalakshmi.girish@hpe.com',
  'name': 'rajalakshmi-ganesan'},
 'reviewers': [{'accountId': '97',
   'avatar': None,
   'email': 'daryl.walleck@rackspace.com',
   'name': 'Daryl Walleck'},
  {'accountId': '7',
   'avatar': None,
   'email': 'jaypipes@gmail.com',
   'name': 'Jay Pipes'}],
 'subProject': 'openstack/tempest',
 'timestamp': 1333428006000}

In [114]:
# read a single feature of an element
data[0]['reviewers']

[{'accountId': '97',
  'avatar': None,
  'email': 'daryl.walleck@rackspace.com',
  'name': 'Daryl Walleck'},
 {'accountId': '7',
  'avatar': None,
  'email': 'jaypipes@gmail.com',
  'name': 'Jay Pipes'}]

## Cleaning data

In [115]:
# Extract and save desired features from each pull request. The new desired data is new_data
new_data = []

for i in range(0, len(data)):
  elem_dict = {"changeId":data[i]['changeId'], "reviewers":data[i]['reviewers'], "owner":data[i]['owner'], "filePaths":data[i]['filePaths'], "time":data[i]['timestamp']}
  new_data.append(elem_dict)
  # print(new_data[i])

print(len(new_data))

6545


In [116]:
new_data[0]

{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d',
 'filePaths': [{'location': 'tempest/tests/test_volumes_get.py'}],
 'owner': {'accountId': '2238',
  'avatar': None,
  'email': 'rajalakshmi.girish@hpe.com',
  'name': 'rajalakshmi-ganesan'},
 'reviewers': [{'accountId': '97',
   'avatar': None,
   'email': 'daryl.walleck@rackspace.com',
   'name': 'Daryl Walleck'},
  {'accountId': '7',
   'avatar': None,
   'email': 'jaypipes@gmail.com',
   'name': 'Jay Pipes'}],
 'time': 1333428006000}

In [117]:
# Remove remaining undesired features. The final desired information is saved in final_data
final_data = []

for i in range(0, len(new_data)):
  elem_dict = {"time":new_data[i]['time'], "changeId":new_data[i]['changeId'], "ownerId": new_data[i]['owner']['accountId'], "reviewersId":[], "filePaths":[]}

  for j in range(0, len(new_data[i]['reviewers'])):
    elem_dict["reviewersId"].append(new_data[i]['reviewers'][j]['accountId'])

  for j in range(0, len(new_data[i]['filePaths'])):
    elem_dict["filePaths"].append(new_data[i]['filePaths'][j]['location'])
  
  final_data.append(elem_dict)
  # print(final_data[i])

print(len(final_data))

6545


In [118]:
final_data[0]

{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d',
 'filePaths': ['tempest/tests/test_volumes_get.py'],
 'ownerId': '2238',
 'reviewersId': ['97', '7'],
 'time': 1333428006000}

In [119]:
# change timestamp value from int to date format
from datetime import datetime

for pr in final_data:
  timestamp = int(pr['time']/1000)  # removing the 3 last zero digits 
  dt_obj = datetime.fromtimestamp(timestamp) 
  pr['time'] = dt_obj

In [120]:
final_data[0]

{'changeId': 'I9e7b9a2491d6a3f01d71551e4cadcccf154d992d',
 'filePaths': ['tempest/tests/test_volumes_get.py'],
 'ownerId': '2238',
 'reviewersId': ['97', '7'],
 'time': datetime.datetime(2012, 4, 3, 4, 40, 6)}

In [121]:
# sort data by time
sorted_data = sorted(final_data, key=lambda d: d['time'])

# Get the Time of available data from selected project
print("Start date:", sorted_data[0]['time'])
print("End date:", sorted_data[-1]['time'])

Start date: 2011-07-18 15:43:34
End date: 2012-05-30 21:39:57


The openstack data that we use in here, containes about 10 months information of openstack project (2011-2012)

In [122]:
type(sorted_data)

list

In [123]:
# Convert list to json
# It should be a string before write it to file
# "default" is added to dumps to make anything not serializable for JSON to string format (like datatime) 
json_string = json.dumps(sorted_data, default=str) 

# Save OpenStack new dataset
file = open('openstack_final.json', 'w')
file.write(json_string)
file.close()

In [124]:
# Move OpenStack dataset to proper location
import shutil

src = "./openstack_final.json"
dst = "/content/drive/MyDrive/Colab Notebooks/My Project/openstack_final.json"
shutil.move(src, dst)

'/content/drive/MyDrive/Colab Notebooks/My Project/openstack_final.json'

# Approach

Should run the code each time from this part

## Load final data and split

In [3]:
# Load final data from drive
f = open('/content/drive/MyDrive/Colab Notebooks/My Project/openstack_final.json',)
data = json.load(f)  # returns json object as a dictionary
for i in range(5):   # 5 first elements of the json list
    print(data[i])
f.close()

{'time': '2011-07-18 15:43:34', 'changeId': 'Ibb2f047c9c45d011361d253d4444b2fd1ebd3612', 'ownerId': '1', 'reviewersId': ['2'], 'filePaths': ['modules/ssh/files/sshd_config', 'manifests/server.pp', 'modules/ssh/manifests/init.pp']}
{'time': '2011-07-18 16:56:22', 'changeId': 'Iea32ad3aedbee996db53655842061d14c9e3d876', 'ownerId': '1', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/manifests/jenkinsuser.pp', 'modules/jenkins_slave/files/known_hosts']}
{'time': '2011-07-18 17:14:05', 'changeId': 'Ib6af08a6af71a6e5853d958a12f63c381eafa039', 'ownerId': '1', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/manifests/jenkinsuser.pp']}
{'time': '2011-07-20 14:37:53', 'changeId': 'Ide7975c522b6b31a23af67b135135c07d75d9bde', 'ownerId': '2', 'reviewersId': ['1'], 'filePaths': ['modules/jenkins_slave/files/pubring.gpg', 'modules/jenkins_slave/manifests/jenkinsuser.pp']}
{'time': '2011-07-20 15:12:47', 'changeId': 'I28119ac1735127812866a4b0519368a829e2e666', 'ownerId': '2', 'r

In [4]:
# convert times to datetime format
from dateutil import parser

for pr in data:
  pr['time'] = parser.parse(pr['time'])

data[0]

{'changeId': 'Ibb2f047c9c45d011361d253d4444b2fd1ebd3612',
 'filePaths': ['modules/ssh/files/sshd_config',
  'manifests/server.pp',
  'modules/ssh/manifests/init.pp'],
 'ownerId': '1',
 'reviewersId': ['2'],
 'time': datetime.datetime(2011, 7, 18, 15, 43, 34)}

In [5]:
# Number of files in whole data
all_files = set()   # make a "set" of all file paths
max_file = 0

for pr in data:
  num_file = 0
  for f in pr['filePaths']:
    all_files.add(f)
    num_file += 1
  if num_file > max_file:
    max_file = num_file

print("Total number of files in dataset:", len(all_files))
print("Maximum number of files in a PR:", max_file)

Total number of files in dataset: 11421
Maximum number of files in a PR: 1757


In [6]:
# Number of reviewers in whole data
all_reviewers = set()   # make a set of all reviewers
for pr in data:
  for r in pr['reviewersId']:
    all_reviewers.add(r)

len(all_reviewers)

82

In [7]:
# Number of developers in whole data
all_devs = set()   # make a "set" of all developers
for pr in data:
  all_devs.add(pr['ownerId'])

len(all_devs)

324

In [8]:
# Split data for train/test sets
init_train = data[0:5900]     # 90% of data is for initial train set (5900 PR)
total_test = data[5900:6545]  # 10% of data will be tested in the whole examination (645 PR)

## Main components of the approach
Extract 3 interaction metrices

In [9]:
# temporary
train = init_train
test = data[5900]

In [10]:
test

{'changeId': 'I38787d4c3e3f2896fdc349d6f7817d3ef78cc2a5',
 'filePaths': ['nova/tests/test_migrations.py'],
 'ownerId': '2835',
 'reviewersId': ['616', '100'],
 'time': datetime.datetime(2012, 5, 3, 21, 32, 38)}

### Reviewer's **expertise** model

#### step by step

In [18]:
files = set()   # make a "set" of file paths. we selected set to ensure not collecting repeated files

for f in test['filePaths']:
  files.add(f)

len(files)

1

In [19]:
all_reviewers = set()   # make a set of all reviewers
for pr in train:
  for r in pr['reviewersId']:
    all_reviewers.add(r)

len(all_reviewers)

78

In [20]:
# create a dataframe that its first column is all existing files
df = pd.DataFrame(files)
df.head()

,0
0,nova/tests/test_migrations.py


In [21]:
df = df.rename(columns={0:'file'})
df.head()

,file
0,nova/tests/test_migrations.py


In [22]:
# add a column of zeros for each existing reviewer to dataframe
for r in all_reviewers:
  df[r] = 0

df.head()

,file,494,1603,97,1179,2,161,1,360,70,1247,2325,512,452,2243,2284,1228,335,1456,475,6,1009,2035,236,2472,5,543,964,642,1132,1549,2622,917,1192,100,308,1216,321,1205,1155,706,681,941,330,490,1561,1998,704,1030,518,385,970,688,8,679,845,693,2230,616,2458,191,2828,927,209,261,24,67,357,447,1816,1689,979,995,233,34,2170,4,112,7
0,nova/tests/test_migrations.py,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
# fill the values of dataframe by checking PRs one by one
for f in files:
  for pr in train:
    if f in pr['filePaths']:
      for r in pr['reviewersId']:
        df.loc[(df['file'] == f), r] += 1

df.head()

,file,494,1603,97,1179,2,161,1,360,70,1247,2325,512,452,2243,2284,1228,335,1456,475,6,1009,2035,236,2472,5,543,964,642,1132,1549,2622,917,1192,100,308,1216,321,1205,1155,706,681,941,330,490,1561,1998,704,1030,518,385,970,688,8,679,845,693,2230,616,2458,191,2828,927,209,261,24,67,357,447,1816,1689,979,995,233,34,2170,4,112,7
0,nova/tests/test_migrations.py,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,0,0,0,0,0,2,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
df = df.set_index('file')
df.head()

,494,1603,97,1179,2,161,1,360,70,1247,2325,512,452,2243,2284,1228,335,1456,475,6,1009,2035,236,2472,5,543,964,642,1132,1549,2622,917,1192,100,308,1216,321,1205,1155,706,681,941,330,490,1561,1998,704,1030,518,385,970,688,8,679,845,693,2230,616,2458,191,2828,927,209,261,24,67,357,447,1816,1689,979,995,233,34,2170,4,112,7
file,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
nova/tests/test_migrations.py,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2,0,0,0,0,0,2,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
# convert the dataframe into matrix
FR = df.to_numpy()
FR.shape

(1, 78)

#### All together
as 'cal_expertise' function

In [26]:
# Function to extract 'Expertise' metrix
# Returns P*M matrix named FR
def cal_expertise(train, test):

  # create a "set" of file paths in test PR. we selected set to ensure not collecting repeated files
  files = set()  
  for f in test['filePaths']:
    files.add(f) 
  print("P is:", len(files))

  # create a set of all reviewers
  all_reviewers = set()   
  for pr in train:
    for r in pr['reviewersId']:
      all_reviewers.add(r)
  print("M is:", len(all_reviewers))

  # create a dataframe that its first column is files in test PR
  df = pd.DataFrame(files)  
  df = df.rename(columns={0:'file'})  # set a name for this column

  # add a column of zeros for each existing reviewer to dataframe
  for r in all_reviewers:
    df[r] = 0

  # fill the cells of the dataframe by checking PRs one by one
  for f in files:
    for pr in train:
      if f in pr['filePaths']:
        for r in pr['reviewersId']:
          df.loc[(df['file'] == f), r] += 1

  # convert the dataframe into matrix
  df = df.set_index('file')
  FR = df.to_numpy()
  print("Shape of FR matrix is:", FR.shape)
  return FR

### Reviewer‑developer **collaboration** model

#### step by step

In [11]:
all_devs = set()   # make a "set" of all developers
for pr in train:
  all_devs.add(pr['ownerId'])

len(all_devs)

305

In [12]:
len(all_reviewers)

82

In [13]:
# create a dataframe that its first column is all existing developers
df = pd.DataFrame(all_devs)
df = df.rename(columns={0:'developer'})
df.head()

,developer
0,494
1,1603
2,97
3,866
4,2459


In [14]:
# add a column of zeros for each existing reviewer to dataframe
for r in all_reviewers:
  df[r] = 0

df.head()

,developer,494,1603,97,866,1179,2,161,1,360,70,1247,2325,512,452,2243,2284,1228,335,1456,475,6,1009,2035,236,2472,5,543,964,642,1132,1549,2622,917,838,1192,100,612,308,1216,...,706,681,941,1699,330,490,1561,1998,704,1030,518,385,970,688,8,679,845,693,2230,616,2458,191,2828,927,209,261,24,67,357,447,1816,1689,979,995,233,34,2170,4,112,7
0,494,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1603,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,97,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,866,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2459,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
# fill the values of dataframe by checking PRs one by one
for pr in train:
  for r in pr['reviewersId']:
    df.loc[(df['developer'] == pr['ownerId']), r] += len(pr['filePaths'])

df.head()

,developer,494,1603,97,866,1179,2,161,1,360,70,1247,2325,512,452,2243,2284,1228,335,1456,475,6,1009,2035,236,2472,5,543,964,642,1132,1549,2622,917,838,1192,100,612,308,1216,...,706,681,941,1699,330,490,1561,1998,704,1030,518,385,970,688,8,679,845,693,2230,616,2458,191,2828,927,209,261,24,67,357,447,1816,1689,979,995,233,34,2170,4,112,7
0,494,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,16,17,3
1,1603,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,...,0,0,0,0,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0
2,97,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,192
3,866,25,0,0,0,0,2,0,0,2,0,0,0,0,0,0,4,0,0,0,0,26,0,0,0,0,4,0,1,0,8,0,0,0,0,0,0,0,0,6,...,0,0,0,0,15,0,0,0,0,0,0,0,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,0,0,0,8,0,0,0,5,35,0
4,2459,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
df = df.set_index('developer')
# convert the dataframe into matrix
DR = df.to_numpy()
DR.shape

(305, 82)

#### All together
as 'cal_collaboration' function

In [17]:
# Function to extract 'Collaboration' metrix
# Returns N*M matrix named DR
def cal_collaboration(train, test):

  # create a "set" of all developers
  all_devs = set()  
  for pr in train:
    all_devs.add(pr['ownerId'])
  print("N is:", len(all_devs))

  # There is a set of all reviewers
  print("M is:", len(all_reviewers))

  # create a dataframe that its first column is all existing developers
  df = pd.DataFrame(all_devs)
  df = df.rename(columns={0:'developer'})

  # add a column of zeros for each existing reviewer to dataframe
  for r in all_reviewers:
    df[r] = 0
  
  # fill the values of dataframe by checking PRs one by one
  for pr in train:
    for r in pr['reviewersId']:
      df.loc[(df['developer'] == pr['ownerId']), r] += len(pr['filePaths'])

  # convert the dataframe into matrix
  df = df.set_index('developer')
  DR = df.to_numpy()
  print("Shape of DR matrix is:", DR.shape)
  return DR

### **Availability** model

#### step by step

In [27]:
len(all_reviewers)

78

In [28]:
test['time']

datetime.datetime(2012, 5, 3, 21, 32, 38)

In [29]:
import datetime as DT

# seven day before the time of our test PR
week_ago = test['time'] - DT.timedelta(days=7)
week_ago

datetime.datetime(2012, 4, 26, 21, 32, 38)

In [30]:
# Extract a list of PRs that are between time of seven days ago up to the time of test PR
week_pr = []

for pr in train:
  if pr['time'] > week_ago and pr['time'] < test['time']:
    week_pr.append(pr)

len(week_pr)

214

In [31]:
# Extract the availability matrix named 'A' that is 1*M
A = []

for r in all_reviewers:
  A.append(0)
  for pr in week_pr:
    if r in pr['reviewersId']:
      A[-1] += len(pr['filePaths'])

len(A)

78

#### All together
as 'cal_availability' function

In [32]:
# Function to extract 'Availability' vector
# Returns 1*M matrix named A 
def cal_availability(train, test):

  # seven day before the time of our test PR
  week_ago = test['time'] - DT.timedelta(days=7)
  
  # Extract a list of PRs that are between time of seven days ago up to the time of test PR
  week_pr = []
  for pr in train:
    if pr['time'] > week_ago and pr['time'] < test['time']:
      week_pr.append(pr)
  print("Num of PRs in last 7 days:", len(week_pr))

  # Extract the availability matrix named 'A' that is 1*M
  A = []
  for r in all_reviewers:
    A.append(0)
    for pr in week_pr:
      if r in pr['reviewersId']:
        A[-1] += len(pr['filePaths'])
  
  print("Size of A matrix is:", len(A))
  return A

### **File-Developer** matrix

#### step by step

In [33]:
len(all_devs)

305

In [34]:
len(files)

1

In [35]:
# create a dataframe that its first column is all existing developers
df = pd.DataFrame(all_devs)
df = df.rename(columns={0:'developer'})
df.head()

,developer
0,494
1,1603
2,97
3,866
4,2459


In [36]:
# add a column of zeros for each existing file in test PR
for f in files:
  df[f] = 0

df.head()

,developer,nova/tests/test_migrations.py
0,494,0
1,1603,0
2,97,0
3,866,0
4,2459,0


In [37]:
# Fill the dataframe with number of times each developer worked on each file
for pr in train:
  for f in pr['filePaths']:
    if f in files:
      df.loc[(df['developer'] == pr['ownerId']), f] += 1

df[df['nova/tests/test_migrations.py']>0]

,developer,nova/tests/test_migrations.py
13,360,1
48,2750,1
83,2245,2
131,100,1
191,1030,1
200,708,1
205,385,1
261,67,1


In [38]:
df = df.set_index('developer')
# convert the dataframe into matrix
FD = df.to_numpy()
FD.shape

(305, 1)

#### All together
as 'cal_fileDev' function

In [39]:
# Function to extract File-Developer matrix
# Returns N*P matrix named FD 
def cal_fileDev(train, test):

  # There is a set of all developers
  print("N is:", len(all_devs))

  # There is a set of files in PR
  print("P is:", len(files))

  # create a dataframe that its first column is all existing developers
  df = pd.DataFrame(all_devs)
  df = df.rename(columns={0:'developer'})

  # add a column of zeros for each existing file in test PR
  for f in files:
    df[f] = 0
  
  # Fill the dataframe with number of times each developer worked on each file
  for pr in train:
    for f in pr['filePaths']:
      if f in files:
        df.loc[(df['developer'] == pr['ownerId']), f] += 1

  # convert the dataframe into matrix
  df = df.set_index('developer')
  FD = df.to_numpy()
  print("Shape of FD matrix is:", FD.shape)
  return FD

## Problem Formulation

### Solution representation

In [84]:
# Gene creation
# Reviewer-File matrix named S with shape of P*M 
P = len(files)  # number of files in test PR
M = len(all_reviewers) # number of all candidate reviewers

# create a dataframe that its first column is all files in test PR
df = pd.DataFrame(files)
df = df.rename(columns={0:'file'})

# add a column of zeros for each existing reviewer to dataframe
for r in all_reviewers:
  df[r] = 0

# convert the dataframe into matrix
df = df.set_index('file')
S = df.to_numpy()

# assign random number to each cell of the dataframe
# each row must have all the integer numbers from 0 to M (distinct numbers)
for row in range(0, P):
  num_of_zeros = random.randint(0, M+1)
  print("number of zero ranks in row", row, "is:", num_of_zeros)
  rand_ranks = num_of_zeros * [0]
  rand_ranks.extend(random.sample(range(1, M-num_of_zeros+1), M-num_of_zeros))  # sampling without replacement
  print("number of non-zero ranks in row", row, "is:", max(rand_ranks))
  random.shuffle(rand_ranks)   # Shuffle zeros with non-zero ranks
  S[row] = rand_ranks

S

number of zero ranks in row 0 is: 44
number of non-zero ranks in row 0 is: 34


array([[14, 12,  7,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0, 16,
         1, 32,  0,  0,  6,  0,  0,  0, 29,  0, 31,  0, 20,  0, 11,  9,
         0, 19,  0,  0, 15, 26, 17,  0, 34,  0,  8,  2,  0,  0,  0,  4,
        23, 28, 22, 21, 24,  0,  0,  5, 33,  0, 30, 18, 25,  0,  0,  0,
         0,  0,  0, 10,  0,  0, 27,  0,  0, 13,  0,  0,  0,  0]])

### Fitness functions

#### Availability

## Test

In [256]:
for test_num in range(0, 1):  # number of examination can be 0 to 644
  # creating test set
  test = total_test[test_num]   # one PR is test in each examination
  # creating train set
  if test_num == 0:
    train = init_train
  else:
    train.append(total_test[test_num-1])
  
  # Do other calculations
  # extract 3 metrices
  FR = cal_expertise(train, test)
  # print(FR)
  print("------------------------------")

  DR = cal_collaboration(train, test)
  # print(DR)
  print("------------------------------")

  A = cal_availability(train, test)
  # print(A)
  print("------------------------------")

  FD = cal_fileDev(train, test)
  # print(FD)

  # do genetic
  # ...



P is: 1
M is: 78
Shape of FR matrix is: (1, 78)
[[0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 3 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0
  0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 2 0 0
  0 0 0 0 0 0]]
------------------------------
N is: 305
M is: 78
Shape of DR matrix is: (305, 78)
[[0 0 8 ... 0 0 0]
 [9 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [2 0 3 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 6 ... 0 0 0]]
------------------------------
Num of PRs in last 7 days: 214
Size of A matrix is: 78
[35, 0, 47, 12, 0, 12, 15, 4, 3, 0, 0, 0, 6, 2, 6, 7, 83, 0, 0, 0, 7, 10, 54, 51, 0, 13, 0, 67, 0, 4, 0, 106, 9, 10, 51, 14, 25, 17, 0, 39, 0, 0, 0, 1, 0, 0, 0, 328, 0, 48, 4, 181, 0, 0, 13, 6, 0, 1, 11, 25, 2, 26, 0, 1, 0, 157, 0, 0, 2, 18, 0, 2, 6, 31, 5, 0, 30, 5]
------------------------------
N is: 305
P is: 1
Shape of FD matrix is: (305, 1)
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
